Main Point of this Notebook:
You can train from where you left off as long as you do not shutdown your kernel. There is no need to save and load weight here.

In [1]:
# Loading Common Packages for Our Team
import pickle as pickle
import numpy as np
import os
import pandas as pd
import random
from tqdm import tqdm
# import xgboost as xgb   # Fan Fei does not have XGBoost on Windows
import pickle as pickle
import cv2
import datetime as dt
print("------Imports complete-----")

------Imports complete-----


In [2]:
# Import Model Specific packages
from keras.preprocessing.image import img_to_array, load_img

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

from multiprocessing import Pool, cpu_count

# callback for saving models
from keras.callbacks import ModelCheckpoint

# os.chdir('C:/deep_learning') # This is where the input dataset is stored
# os.getcwd()

Using TensorFlow backend.


In [3]:
random_seed = 987654321
random.seed(random_seed)
np.random.seed(random_seed)

# train_path = "c:/deep_learning/kaggle/amazon/train/"
# test_path = "c:/deep_learning/kaggle/amazon/test/"
train_path = "f:/kaggle/amazon/train-jpg/"
test_path = "f:/kaggle/amazon/test-jpg/"
# train_path = "/mnt/f/kaggle/amazon/train-jpg/"
# test_path = "/mnt/f/kaggle/amazon/test-jpg/"

train = pd.read_csv("c:/deep_learning/kaggle/amazon/train_v2.csv")
# train = pd.read_csv("c:/deep_learning/kaggle/amazon/train_v2_short.csv")
test =  pd.read_csv("c:/deep_learning/kaggle/amazon/sample_submission_v2.csv")
# test =  pd.read_csv("c:/deep_learning/kaggle/amazon/sample_submission_v2_short.csv")

#Location for saving serialized objects
obj_save_path = "c:/deep_learning/kaggle/amazon/model/"
subm_output_path = "c:/deep_learning/kaggle/amazon/submissions/"
print("------Paths Built-----")

------Paths Built-----


In [4]:
def keras_image_prep(df, data_path, name, dim, labels):
    x_out = []
    y_out = []

    # flatten = lambda l: [item for sublist in l for item in sublist]
    # labels = list(set(flatten([l.split(' ') for l in df['tags'].values])))

    label_map = {l: i for i, l in enumerate(labels)}
    # inv_label_map = {i: l for l, i in label_map.items()}

    for image_name, tags in tqdm(df.values, miniters=1000):
        img = img_to_array(load_img(data_path + image_name + '.jpg', target_size=[dim,dim]))
        targets = np.zeros(17)
        for t in tags.split(' '):
            targets[label_map[t]] = 1 
        x_out.append(img)
        y_out.append(targets)
    
    x_out = np.array(x_out, np.float16) / 255.
    y_out = np.array(y_out, np.uint8)

    print(x_out.shape)
    print(y_out.shape)
    return x_out, y_out

In [5]:
input_dim = 32
labels = train['tags'].str.get_dummies(sep=' ').columns

x_train, y_train = keras_image_prep(train, train_path, 'train', input_dim, labels)

# this is to provide a local validation F-score
split = 35000
# split = 80

x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(input_dim, input_dim, 3)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])
              
filepath= obj_save_path + "weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# the explicit split approach was taken so as to allow for a local validation
# model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, callbacks=callbacks_list, verbose=0)

# Fit the model
model.fit(x_train, y_train,
          batch_size=128,
          epochs=4,
          verbose=1,
          callbacks=callbacks_list,
          validation_data=(x_valid, y_valid))

100%|████████████████████████████████████████████████████████████████████████████| 40479/40479 [13:15<00:00, 34.73it/s]


(40479, 32, 32, 3)
(40479, 17)
Train on 35000 samples, validate on 5479 samples
Epoch 1/4
35000/35000 [==============================] - 314s - loss: 0.2525 - acc: 0.9040 - val_loss: 0.1995 - val_acc: 0.9187
Epoch 2/4
35000/35000 [==============================] - 328s - loss: 0.2041 - acc: 0.9201 - val_loss: 0.1826 - val_acc: 0.9279
Epoch 3/4
35000/35000 [==============================] - 336s - loss: 0.1903 - acc: 0.9252 - val_loss: 0.1766 - val_acc: 0.9282
Epoch 4/4
35000/35000 [==============================] - 345s - loss: 0.1833 - acc: 0.9283 - val_loss: 0.1677 - val_acc: 0.9329


In [6]:
# Local Validation Results
from sklearn.metrics import fbeta_score
p_valid = model.predict(x_valid, batch_size=128,verbose=2)
print(y_valid.shape)
print(p_valid.shape)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

(5479, 17)
(5479, 17)
0.837609804454


In [8]:
# purge the x_train and y_train dataset, and read in the test images
x_train = []
y_train = []
x_test, y_test = keras_image_prep(test, test_path, 'test', input_dim, labels)

# predict using the trained model
y_test = model.predict(x_test,batch_size=128,verbose=2)
y_c_test = (y_test > .222222)*1.0        # before multiplying by 1.0, this appears as an array of True and False
print(y_c_test)


100%|████████████████████████████████████████████████████████████████████████████| 61191/61191 [22:55<00:00, 44.49it/s]


(61191, 32, 32, 3)
(61191, 17)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  1.]]


In [9]:
# Build Submission, using label outputted from long time ago
test_labels = []
for row in range(y_c_test.shape[0]):
    test_labels.append(' '.join(labels[y_c_test[row,:]==1]))
Submission_PDFModel = test.copy()
Submission_PDFModel.drop('tags', axis = 1)
Submission_PDFModel['tags'] = test_labels
Submission_PDFModel.to_csv(str(subm_output_path + 'sub_simple_keras_32x32_rd1.csv'), index = False)

In [10]:
x_train, y_train = keras_image_prep(train, train_path, 'train', input_dim, labels)
# Fit the model
model.fit(x_train, y_train,
          batch_size=128,
          epochs=4,
          verbose=1,
          callbacks=callbacks_list,
          validation_data=(x_valid, y_valid))
p_valid = model.predict(x_valid, batch_size=128,verbose=2)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

100%|████████████████████████████████████████████████████████████████████████████| 40479/40479 [17:09<00:00, 47.16it/s]


(40479, 32, 32, 3)
(40479, 17)
Train on 40479 samples, validate on 5479 samples
Epoch 1/4
40479/40479 [==============================] - 346s - loss: 0.1778 - acc: 0.9306 - val_loss: 0.1671 - val_acc: 0.9331
Epoch 2/4
40479/40479 [==============================] - 346s - loss: 0.1737 - acc: 0.9323 - val_loss: 0.1606 - val_acc: 0.9356
Epoch 3/4
40479/40479 [==============================] - 344s - loss: 0.1694 - acc: 0.9336 - val_loss: 0.1544 - val_acc: 0.9386
Epoch 4/4
40479/40479 [==============================] - 622s - loss: 0.1656 - acc: 0.9348 - val_loss: 0.1532 - val_acc: 0.9386
0.853080735294


In [11]:
# predict using the trained model
y_test = model.predict(x_test,batch_size=128,verbose=2)
y_c_test = (y_test > .222222)*1.0        # before multiplying by 1.0, this appears as an array of True and False

# Build Submission, using label outputted from long time ago
test_labels = []
for row in range(y_c_test.shape[0]):
    test_labels.append(' '.join(labels[y_c_test[row,:]==1]))
Submission_PDFModel = test.copy()
Submission_PDFModel.drop('tags', axis = 1)
Submission_PDFModel['tags'] = test_labels
Submission_PDFModel.to_csv(str(subm_output_path + 'sub_simple_keras_32x32_rd2.csv'), index = False)

In [12]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=4,
          verbose=1,
          callbacks=callbacks_list,
          validation_data=(x_valid, y_valid))
p_valid = model.predict(x_valid, batch_size=128,verbose=2)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

Train on 40479 samples, validate on 5479 samples
Epoch 1/4
40479/40479 [==============================] - 363s - loss: 0.1642 - acc: 0.9354 - val_loss: 0.1608 - val_acc: 0.9350
Epoch 2/4
40479/40479 [==============================] - 352s - loss: 0.1615 - acc: 0.9364 - val_loss: 0.1452 - val_acc: 0.9415
Epoch 3/4
40479/40479 [==============================] - 347s - loss: 0.1578 - acc: 0.9376 - val_loss: 0.1447 - val_acc: 0.9421
Epoch 4/4
40479/40479 [==============================] - 353s - loss: 0.1562 - acc: 0.9383 - val_loss: 0.1413 - val_acc: 0.9431
0.863012723817


In [13]:
# predict using the trained model
y_test = model.predict(x_test,batch_size=128,verbose=2)
y_c_test = (y_test > .222222)*1.0        # before multiplying by 1.0, this appears as an array of True and False

# Build Submission, using label outputted from long time ago
test_labels = []
for row in range(y_c_test.shape[0]):
    test_labels.append(' '.join(labels[y_c_test[row,:]==1]))
Submission_PDFModel = test.copy()
Submission_PDFModel.drop('tags', axis = 1)
Submission_PDFModel['tags'] = test_labels
Submission_PDFModel.to_csv(str(subm_output_path + 'sub_simple_keras_32x32_rd3.csv'), index = False)

In [ ]:
# Trying out sampling of DataFrame
test_short = test.sample(frac=0.25,replace=False)
print(test_short.shape)

In [ ]:
print(labels)
print(Submission_PDFModel.shape)